In [1]:
from load_database import load_vector_store_local

vecstore = load_vector_store_local("sec-10-K","10-K")

/home/athekunal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

In [2]:
import chromadb

query_results = vecstore.query(
    query_texts="What are the risk factors of Apple for 2022?",
    n_results=20,
    where={'full_metadata':'AAPL_2022_RISK_FACTORS_10-K'},
    include=["metadatas","documents","distances","embeddings"]
)

In [3]:
len(query_results['embeddings'][0])

20

In [4]:
query_results.keys()

dict_keys(['ids', 'embeddings', 'documents', 'metadatas', 'distances'])

In [5]:
from chromadb.utils import embedding_functions


default_ef = embedding_functions.DefaultEmbeddingFunction()

query_embed = default_ef(texts="What are the risk factors of Apple for 2022?")

In [6]:
# query_embed

In [14]:
llm1_output_dict = {'Section_Names': ['RISK_FACTORS',
  'MARKET_RISK_DISCLOSURES',
  'LEGAL_PROCEEDINGS',
  'UNRESOLVED_STAFF_COMMENTS',
  'MINE_SAFETY'],
 'Tickers': ['AAPL', 'TSLA'],
 'Years': ['2021', '2022'],
 'augmented_query': ['What is the risk associated with Apple stock in 2021?',
  'What is the risk associated with Tesla stock in 2021?',
  'What is the risk associated with Apple stock in 2022?',
  'What is the risk associated with Tesla stock in 2022?']}

In [8]:
s = "TSLA_2023_RISK_FACTORS_10-K"

In [9]:
import itertools
section_names = llm1_output_dict['Section_Names']
ticker_names = llm1_output_dict['Tickers']
years = llm1_output_dict['Years']
doc_name = "10-K"
individual_metadata = [elem[0]+"_"+elem[1]+"_"+elem[2]+"_"+doc_name for elem in list(itertools.product(ticker_names,years,section_names))]

In [10]:
individual_metadata

['AAPL_2021_RISK_FACTORS_10-K',
 'AAPL_2021_MARKET_RISK_DISCLOSURES_10-K',
 'AAPL_2021_LEGAL_PROCEEDINGS_10-K',
 'AAPL_2021_UNRESOLVED_STAFF_COMMENTS_10-K',
 'AAPL_2021_MINE_SAFETY_10-K',
 'AAPL_2022_RISK_FACTORS_10-K',
 'AAPL_2022_MARKET_RISK_DISCLOSURES_10-K',
 'AAPL_2022_LEGAL_PROCEEDINGS_10-K',
 'AAPL_2022_UNRESOLVED_STAFF_COMMENTS_10-K',
 'AAPL_2022_MINE_SAFETY_10-K',
 'TSLA_2021_RISK_FACTORS_10-K',
 'TSLA_2021_MARKET_RISK_DISCLOSURES_10-K',
 'TSLA_2021_LEGAL_PROCEEDINGS_10-K',
 'TSLA_2021_UNRESOLVED_STAFF_COMMENTS_10-K',
 'TSLA_2021_MINE_SAFETY_10-K',
 'TSLA_2022_RISK_FACTORS_10-K',
 'TSLA_2022_MARKET_RISK_DISCLOSURES_10-K',
 'TSLA_2022_LEGAL_PROCEEDINGS_10-K',
 'TSLA_2022_UNRESOLVED_STAFF_COMMENTS_10-K',
 'TSLA_2022_MINE_SAFETY_10-K']

In [11]:
from load_database import load_vector_store_local
from utils import maximal_marginal_relevance
vecstore = load_vector_store_local("sec-10-K","10-K")

In [11]:
from chromadb.utils import embedding_functions
query = "What are the risk Factors of Apple and Tesla for the year 2022 and 2021?"


default_ef = embedding_functions.DefaultEmbeddingFunction()

query_embed = default_ef(texts=query)
# query_embed

In [12]:
import numpy as np
query_embed_arr = np.array(query_embed[0])
query_embed_arr.shape

(384,)

In [28]:
from collections import defaultdict
all_relevant_docs = defaultdict()
for tic in ticker_names:
    for yr in years:
        individual_metadata = [{"full_metadata":elem[0]+"_"+elem[1]+"_"+elem[2]+"_"+doc_name} for elem in list(itertools.product([tic],[yr],section_names))]
        where_clause = {"$or":individual_metadata}
        query_results = vecstore.query(
        query_texts=query,
        n_results=20,
        where=where_clause,
        include=["metadatas","documents","distances","embeddings"]
        )
        idxs = maximal_marginal_relevance(query_embed_arr,query_results['embeddings'][0],k=5)
        relevant_docs = [query_results['documents'][0][i] for i in idxs]
        relevant_metadata = [ query_results['metadatas'][0][i] for i in idxs]
        print(len(relevant_docs),len(relevant_metadata))
        all_relevant_docs[f"{tic}_{yr}"] = {"docs":relevant_docs,"metadata":relevant_metadata}

5 5
5 5
5 5
5 5


In [18]:
all_relevant_docs.keys()

dict_keys(['AAPL_2021', 'AAPL_2022', 'TSLA_2021', 'TSLA_2022'])

In [43]:
# query_results['embeddings'][0]

In [44]:
idxs = maximal_marginal_relevance(query_embed_arr,query_results['embeddings'][0],k=5)
idxs

[12, 14, 19, 11, 7]

In [53]:
relevant_docs = [query_results['documents'][0][i] for i in idxs]
relevant_metadata = [ query_results['metadatas'][0][i] for i in idxs]

In [54]:
relevant_metadata

[{'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'},
 {'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'},
 {'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'},
 {'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'},
 {'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'}]

In [24]:
for k,v in all_relevant_docs.items():
    print(k,v)
    break

AAPL_2021 {'docs': 'The Company’s business, results of operations and financial condition, as well as the price of the Company’s stock, have been adversely affected and could in the future be materially adversely affected by the COVID-19 pandemic.\n COVID-19 has had, and continues to have, a significant impact around the world, prompting governments and businesses to take unprecedented measures in response. The Company’s minority market share in the global smartphone, personal computer and tablet markets can make developers less inclined to develop or upgrade software for the Company’s products and more inclined to devote their resources to developing and upgrading software for competitors’ products with larger market share.\n When developers focus their efforts on these competing platforms, the availability and quality of applications for the Company’s devices can suffer. These new ventures are inherently risky and may not be successful.\n The failure of any significant investment cou

In [30]:
def get_relevant_docs_via_mmr(relevant_docs_dict:dict):
    relevant_sentences = ""

    for key,value in relevant_docs_dict.items():
        tic,year = key.split("_")
        relevant_sentences+=f"Relevant documents for {tic} in the year {year}"+": "
        print(value)
        relevant_sentences+="\n".join(value['docs'])
        relevant_sentences+="\n\n"
    
    return relevant_sentences

len(get_relevant_docs_via_mmr(all_relevant_docs).split(" "))

{'docs': ['The Company’s business, results of operations and financial condition, as well as the price of the Company’s stock, have been adversely affected and could in the future be materially adversely affected by the COVID-19 pandemic.\n COVID-19 has had, and continues to have, a significant impact around the world, prompting governments and businesses to take unprecedented measures in response.', 'The Company’s minority market share in the global smartphone, personal computer and tablet markets can make developers less inclined to develop or upgrade software for the Company’s products and more inclined to devote their resources to developing and upgrading software for competitors’ products with larger market share.\n When developers focus their efforts on these competing platforms, the availability and quality of applications for the Company’s devices can suffer.', 'These new ventures are inherently risky and may not be successful.\n The failure of any significant investment could 

1271

In [31]:
from utils import key_modifier,modify_all_keys,get_query_metadata,get_relevant_sentences,maximal_marginal_relevance


In [1]:
from load_database import get_relevant_dict_with_mmr,get_relevant_docs_via_mmr
from load_database import load_vector_store_local
from utils import maximal_marginal_relevance
vecstore = load_vector_store_local("sec-10-K","10-K")


/home/athekunal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

In [2]:
from collections import defaultdict
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings
from chromadb.utils import embedding_functions
import numpy as np
import itertools

In [15]:

def get_relevant_dict_with_mmr_(llm1_output_dict,restore_collection,user_query,doc_name="10-K"):
    section_names = llm1_output_dict['Section_Names']
    ticker_names = llm1_output_dict['Tickers']
    years = llm1_output_dict['Years']
    all_relevant_docs = defaultdict()
    default_ef = embedding_functions.DefaultEmbeddingFunction()
    query_embed = default_ef(texts=user_query)
    query_embed_arr = np.array(query_embed[0])

    for tic in ticker_names:
        for yr in years:
            individual_metadata = [{"full_metadata":elem[0]+"_"+elem[1]+"_"+elem[2]+"_"+doc_name} for elem in list(itertools.product([tic],[yr],section_names))]
            print(individual_metadata)
            where_clause = {"$or":individual_metadata}
            print(where_clause)
            query_results = restore_collection.query(
            query_texts=user_query,
            n_results=20,
            where=where_clause,
            include=["metadatas","documents","distances","embeddings"]
            )
            print(query_results)
            idxs = maximal_marginal_relevance(query_embed_arr,query_results['embeddings'][0],k=5)
            relevant_docs = [query_results['documents'][0][i] for i in idxs]
            relevant_metadata = [ query_results['metadatas'][0][i] for i in idxs]
            all_relevant_docs[f"{tic}_{yr}"] = {"docs":relevant_docs,"metadata":relevant_metadata}

    return all_relevant_docs

In [16]:
print(vecstore)

name='SEC-10-K' id=UUID('68fc286d-f5c1-45c4-893b-58f73a6a7e3c') metadata=None


In [8]:
llm1_output_dict = {'Section_Names': ['RISK_FACTORS', 'MARKET RISK DISCLOSURES', 'LEGAL PROCEEDINGS', 'UNRESOLVED STAFF COMMENTS', 'MINE SAFETY'], 'Tickers': ['AAPL'], 'Years': ['2021', '2022'], 'augmented_query': ['Compare the risk associated with Apple stock for the year 2021', 'Compare the risk associated with Apple stock for the year 2022']}
relevant_dict = get_relevant_dict_with_mmr_(llm1_output_dict,vecstore,"Compare the risk associated with Apple stock for the years 2021 and 2022?")
print(relevant_dict)


{'$or': [{'full_metadata': 'AAPL_2021_RISK FACTORS_10-K'}, {'full_metadata': 'AAPL_2021_MARKET RISK DISCLOSURES_10-K'}, {'full_metadata': 'AAPL_2021_LEGAL PROCEEDINGS_10-K'}, {'full_metadata': 'AAPL_2021_UNRESOLVED STAFF COMMENTS_10-K'}, {'full_metadata': 'AAPL_2021_MINE SAFETY_10-K'}]}
{'ids': [[]], 'embeddings': [[]], 'documents': [[]], 'metadatas': [[]], 'distances': [[]]}
{'$or': [{'full_metadata': 'AAPL_2022_RISK FACTORS_10-K'}, {'full_metadata': 'AAPL_2022_MARKET RISK DISCLOSURES_10-K'}, {'full_metadata': 'AAPL_2022_LEGAL PROCEEDINGS_10-K'}, {'full_metadata': 'AAPL_2022_UNRESOLVED STAFF COMMENTS_10-K'}, {'full_metadata': 'AAPL_2022_MINE SAFETY_10-K'}]}
{'ids': [[]], 'embeddings': [[]], 'documents': [[]], 'metadatas': [[]], 'distances': [[]]}
defaultdict(None, {'AAPL_2021': {'docs': [], 'metadata': []}, 'AAPL_2022': {'docs': [], 'metadata': []}})


In [ ]:
relevant_sentences = get_relevant_docs_via_mmr(relevant_dict)
print(relevant_sentences)

In [17]:
# user_query = "Compare the risk associated with Apple stock for the years 2021 and 2022?"
# vecstore.query(
#     query_texts=user_query,
#     n_results=2,
#     include=["metadatas","documents","distances","embeddings"]
#     )

In [20]:
"_".join("RISK_FACTORS".split(" "))

'RISK_FACTORS'

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings

finbert_embed = HuggingFaceEmbeddings(model_name="ProsusAI/finbert")

class FinBertEmbeddings(EmbeddingFunction):
    def __call__(self, texts: Documents) -> Embeddings:
        embed_out = finbert_embed.embed_documents(texts)
        return embed_out

No sentence-transformers model found with name /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
query = "What are the risk factors Apple for the year 2022?"

finbert = FinBertEmbeddings()

In [7]:
import numpy as np
query_embed = finbert(texts=query)
query_embed_arr = np.array(query_embed[0])

In [8]:
query_embed_arr

array([ 1.82527483e-01,  2.10601196e-01, -2.58634448e-01, -2.08617985e-01,
       -9.78663191e-02, -7.12072432e-01,  7.79373944e-01,  6.05757713e-01,
        2.57762700e-01, -1.62297443e-01, -4.09485430e-01, -4.43593144e-01,
        3.94873530e-01,  2.59973437e-01,  1.37500418e-02, -4.95217331e-02,
        3.22281986e-01,  4.67214704e-01,  3.84518653e-01,  7.69835338e-02,
        3.88191104e-01,  1.59892544e-01,  2.74140090e-02,  2.96534538e-01,
       -2.21730903e-01,  7.43004322e-01,  1.94182739e-01, -3.12091261e-01,
        7.63344467e-02,  5.54689407e-01,  6.61128461e-01,  3.79334718e-01,
       -2.87863642e-01,  4.58972663e-01,  5.86117089e-01, -4.76604313e-01,
        1.56925365e-01, -2.43099127e-02, -4.13171202e-01,  8.34424123e-02,
       -7.89192855e-01, -2.88674563e-01,  3.10100198e-01, -5.59933841e-01,
        6.27205595e-02, -7.55767524e-01,  2.88778514e-01, -3.42592783e-02,
       -5.05156994e-01,  5.89336693e-01, -8.60973299e-01, -1.30691871e-01,
       -1.29021347e-01,  